# **importing** **libraries**

In [ ]:
# Core
import pandas as pd
import numpy as np

# Visualization (optional)
import matplotlib.pyplot as plt
import seaborn as sns

# NLP
import re
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

# ML / DL
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam


# **Load** **data**

In [ ]:
train_df = pd.read_csv("/content/train.csv")
test_df  = pd.read_csv("/content/test.csv")

label_cols = [
    "toxic",
    "severe_toxic",
    "obscene",
    "threat",
    "insult",
    "identity_hate"
]

train_df.head()

# clean the text

In [ ]:
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)
    words = text.split()
    return " ".join([w for w in words if w not in stop_words])

train_df["cleaned_comment"] = train_df["comment_text"].apply(clean_text)
test_df["cleaned_comment"]  = test_df["comment_text"].apply(clean_text)

train_df[["comment_text", "cleaned_comment"]].head()

# tokenizing and padding

In [ ]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(train_df["cleaned_comment"])

X = pad_sequences(
    tokenizer.texts_to_sequences(train_df["cleaned_comment"]),
    maxlen=200,
    padding="post"
)

y = train_df[label_cols].values

# data spliting into train and validation

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)


# experiment with cnn,lstm and bert models

# 1st model with CNN

In [ ]:
cnn_model = Sequential([
    Embedding(20000, 128),
    Conv1D(128, 5, activation="relu"),
    GlobalMaxPooling1D(),
    Dense(6, activation="sigmoid")
])

cnn_model.compile(
    optimizer=Adam(0.001),
    loss="binary_crossentropy"
)

cnn_model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=64
)

# 2nd model with LSTM

In [ ]:
lstm_model = Sequential([
    Embedding(20000, 128),
    LSTM(64),
    Dropout(0.5),
    Dense(6, activation="sigmoid")
])

lstm_model.compile(
    optimizer=Adam(0.001),
    loss="binary_crossentropy"
)

lstm_model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=64
)

# CNN and LSTM model evaluation

In [ ]:
def evaluate_model(model, X_val, y_val, name):
    print(f"\n===== {name} Evaluation =====")
    y_pred = (model.predict(X_val) > 0.5).astype(int)

    print(classification_report(
        y_val,
        y_pred,
        target_names=label_cols,
        zero_division=0
    ))

    macro_f1 = f1_score(y_val, y_pred, average="macro")
    print("Macro F1-score:", macro_f1)

In [ ]:
evaluate_model(cnn_model, X_val, y_val, "CNN")
evaluate_model(lstm_model, X_val, y_val, "LSTM")


# Install Transformers Dataset

In [ ]:
!pip install transformers datasets


In [ ]:
!pip uninstall -y transformers accelerate peft
!pip install transformers==4.38.2 accelerate==0.27.2 peft==0.9.0


# DistilBert model

In [ ]:
import pandas as pd
import numpy as np
import torch

from datasets import Dataset
from sklearn.metrics import classification_report, f1_score

from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer
)

# Prepare Dataset

In [ ]:
train_df = pd.read_csv("/content/train.csv")

label_cols = [
    "toxic",
    "severe_toxic",
    "obscene",
    "threat",
    "insult",
    "identity_hate"
]

train_df = train_df[["comment_text"] + label_cols]


In [ ]:
dataset = Dataset.from_pandas(train_df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)


# Tokenization

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased"
)

def tokenize(batch):
    return tokenizer(
        batch["comment_text"],
        truncation=True,
        padding="max_length",
        max_length=64
    )

dataset = dataset.map(tokenize, batched=True)


# Add Labels

In [ ]:
def add_labels(batch):
    batch["labels"] = [
        [float(batch[col][i]) for col in label_cols]
        for i in range(len(batch[label_cols[0]]))
    ]
    return batch

dataset = dataset.map(add_labels, batched=True)


In [ ]:
dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)


# Model & Training

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(label_cols),
    problem_type="multi_label_classification"
)


In [ ]:
training_args = TrainingArguments(
    output_dir="./toxicity_results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,
    logging_steps=500,
    save_strategy="no",
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer
)


In [ ]:
trainer.train()


# Evaluation of DistilBert


In [ ]:
from sklearn.metrics import classification_report, f1_score
import torch

preds = trainer.predict(dataset["test"])

y_true = preds.label_ids
y_prob = torch.sigmoid(torch.tensor(preds.predictions)).numpy()
y_pred = (y_prob > 0.5).astype(int)

print(classification_report(
    y_true,
    y_pred,
    target_names=label_cols,
    zero_division=0
))

print("Macro F1-score:",
      f1_score(y_true, y_pred, average="macro"))


# Save the Final Model

In [ ]:
model.save_pretrained("toxicity_distilbert")
tokenizer.save_pretrained("toxicity_distilbert")

# testing prediction

In [ ]:
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

label_cols = [
    "toxic", "severe_toxic", "obscene",
    "threat", "insult", "identity_hate"
]

tokenizer = DistilBertTokenizerFast.from_pretrained("toxicity_distilbert")
model = DistilBertForSequenceClassification.from_pretrained("toxicity_distilbert")
model.eval()


In [ ]:
def predict_toxicity(text, threshold=0.5):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=64
    )

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits).squeeze().numpy()

    results = {
        label: float(prob)
        for label, prob in zip(label_cols, probs)
    }

    predictions = {
        label: int(prob >= threshold)
        for label, prob in results.items()
    }

    return results, predictions


# Give Input and get Output

In [ ]:
text = "You are a disgusting idiot and should be banned"
scores, preds = predict_toxicity(text)

scores, preds


# Download the Model

In [ ]:
!zip -r toxicity_distilbert.zip toxicity_distilbert


In [ ]:
from google.colab import files
files.download("toxicity_distilbert.zip")
